In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("E://Datasets/titanic.csv")
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [3]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=2)

In [13]:
trf1 = ColumnTransformer(transformers=[
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')



trf2 = ColumnTransformer(transformers=[
    ("ohe_sex_embarked",OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')



trf3 = ColumnTransformer(transformers=[
    ('scale',MinMaxScaler(),slice(0,10))
])

feature_selection = SelectKBest(score_func=chi2,k=8)

dt = DecisionTreeClassifier()



In [14]:
pipe = Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("feature_selection",feature_selection),
    ("decision_tree",dt)
])

In [15]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('feature_selection',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000206BFA15DC0>)),
                ('decision_tree', DecisionTreeClassifier())])

In [16]:
y_pred = pipe.predict(x_test)

In [19]:
accuracy_score(y_test,y_pred)

0.6368715083798883

In [23]:
from sklearn.model_selection import GridSearchCV
params = {
    "decision_tree__max_depth":[1,2,3,4,5,None]
}
grid = GridSearchCV(pipe,params,cv=6,scoring='accuracy')
grid.fit(x_train,y_train)
grid.best_score_

0.623629112662014